In [6]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC, LinearSVC

In [7]:
import imagedataset as imgdat
import bagofvisualwords as bovw

In [8]:
#Get Annotations
annotations = imgdat.GetAnnotations()

In [9]:
#Try to load all descriptors previously generated if not generate one
try:
    all_descriptors = imgdat.LoadAllDescriptors()
except:
    all_descriptors = imgdat.GenerateAllDescriptors(annotations)

Loading all sift descriptors in dataset from file...
All descriptors loaded ! 


In [10]:
#Use standardscaler for all SIFT descriptors before process them with any representation algorithm.
sc_dscs = StandardScaler()
all_descriptors = sc_dscs.fit_transform(all_descriptors)

In [11]:
#Clustering descriptors using KMeans in 120 classes 
try:
    kmeans_bovw = bovw.LoadKmeans()
except:
    n_clusters = 120
    kmeans_bovw = bovw.GenerateKmeans(all_descriptors, n_clusters)

Loading kmeans...
bovw_centers.kmeans Loaded !


In [12]:
#Image representation
try:
    X_bovw = np.load("bovw_X.npy")
    y_bovw = np.load("bovw_y.npy")
except:
    X_bovw, y_bovw = bovw.BovwFeatures(annotations, kmeans_bovw, sc_dscs)
    np.save("bovw_X", X_bovw)
    np.save("bovw_y", y_bovw)

In [13]:
#Classification 
#Dataset splitting into the training set and test set
X_train_bovw, X_test_bovw, y_train_bovw, y_test_bovw = train_test_split(X_bovw, y_bovw, test_size = 0.2, random_state = 0)

In [14]:
#Scaling representation features
sc_repr_bovw = StandardScaler()
X_train_bovw = sc_repr_bovw.fit_transform(X_train_bovw)
X_test_bovw = sc_repr_bovw.transform(X_test_bovw)

C:\Anaconda2\envs\tensorflowproject\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [17]:
#Using a OVR
#One-vs-the-rest (OvR) multiclass/multilabel strategy
clf_ovr = OneVsRestClassifier(SVC(kernel='linear', C=0.3))
clf_ovr.fit(X_train_bovw, y_train_bovw)
print('==========')
print('train score:' + str(clf_ovr.score(X_train_bovw,y_train_bovw)))
print('test score:' + str(clf_ovr.score(X_test_bovw,y_test_bovw)))
y_pred = clf_ovr.predict(X_test_bovw)
y_pred = (y_pred > 0.5)
cm = confusion_matrix(y_test_bovw, y_pred)
print(cm)

train score:0.966921119593
test score:0.838383838384
[[26 12]
 [ 4 57]]
